In [ ]:
import torch
import pandas as pd
import numpy as np
import torch.optim as optim 
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.distributions.one_hot_categorical import OneHotCategorical

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/clean.csv', sep='\t')
train_df.head()  

,Unnamed: 0,reviewerID,asin,reviewText,overall,votes_up,votes_all,label,text_clean,sentiment,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght
0,0,7885,3901,"First off, allow me to correct a common mistak...",5.0,6,7,0,first allow correct common mistake made one re...,0.198494,416,1987,13,4.776442,32.000000
1,1,52087,47978,I am really troubled by this Story and Enterta...,3.0,99,134,0,really troubled story entertained way presente...,0.030000,477,2225,21,4.664570,22.714286
2,2,5701,3667,A near-perfect film version of a downright glo...,4.0,14,14,1,nearperfect film version downright gloomy play...,0.221667,71,357,4,5.028169,17.750000
3,3,47191,40892,Keep your expectations low. Really really low...,1.0,4,7,0,keep expectation low really really low may ent...,-0.106471,117,577,14,4.931624,8.357143
4,4,40957,15367,"""they dont make em like this no more...""well.....",5.0,3,6,0,dont make em like morewell throw phrase window...,0.037500,98,461,21,4.704082,4.666667


In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/clean_test.csv', sep='\t')
test_df.head()  

,Unnamed: 0,Id,reviewerID,asin,reviewText,overall,text_clean,sentiment,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght
0,0,0,82947,37386,I REALLY wanted this series but I am in SHOCK ...,1.0,really wanted series shock price series would ...,0.200000,39,158,4,4.051282,9.750000
1,1,1,10154,23543,I have to say that this is a work of art for m...,4.0,say work art movie production value definitely...,0.296491,169,777,11,4.597633,15.363636
2,2,2,5789,5724,Alien 3 is certainly the most controversal fil...,3.0,alien 3 certainly controversal film alien seri...,0.016857,365,1699,21,4.654795,17.380952
3,3,3,9198,5909,"I love this film...preachy? Well, of course i...",5.0,love filmpreachy well course also lyrical beau...,0.333766,71,326,14,4.591549,5.071429
4,4,4,33252,21214,Even though I previously bought the Gamera Dou...,5.0,even though previously bought gamera double fe...,0.246667,78,384,6,4.923077,13.000000


In [ ]:
bert_train = np.load("/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bert_clean_train.npy")
# bert_test = np.load("/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bert_clean_test.npy")

In [ ]:
small_batch_bert_train = bert_train[:2000]
small_batch_labels = train_df['label'][:2000]
X = small_batch_bert_train
y = small_batch_labels

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2020)

In [ ]:
#一个简单的CNN模型
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.l0 = torch.nn.Dropout(p = 0.2)
    self.l1 = torch.nn.Linear(768, 512)
    self.l2 = torch.nn.Linear(512, 256)
    self.l3 = torch.nn.Linear(256, 128)
    self.l4 = torch.nn.Linear(128, 64)
    self.l5 = torch.nn.Linear(64, 10)

  def forward(self, x):
    x = x.view(-1, 768)
    x = self.l0(x)
    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = F.relu(self.l3(x))
    x = F.relu(self.l4(x))
    return self.l5(x)
model = Net()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda', index=0)

In [ ]:
from torch.utils.data import Dataset, DataLoader
class Mydata(Dataset):
  def __init__(self):
    self.bert_train = np.load("/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bert_clean_train.npy")
    # self.bert_test = np.load("/content/drive/MyDrive/Colab Notebooks/xtzx/hw6/data/bert_clean_test.npy")
    self.labels_train = train_df['label']
    self.train_len = self.bert_train.shape[0]
    # self.test_len = self.bert_test.shape[0]
  
  def __getitem__(self, index):
    assert index < self.train_len
    return self.bert_train[index], self.labels_train[index]
  
  def __len__(self):
    return self.train_len

dataset = Mydata()
dataloader = DataLoader(dataset, batch_size = 32, shuffle = True, num_workers = 2)

In [ ]:
batchsize = 32

class Traindata(Dataset):
  def __init__(self):
    self.xs = x_train
    self.ys = y_train
    self.len = len(x_train)

  def __getitem__(self, index):
    assert index < self.len
    return self.xs[index], self.ys[index]
  
  def __len__(self):
    return self.len

class Testdata(Dataset):
  def __init__(self):
    self.xs = x_test
    self.ys = y_test
    self.len = len(x_test)

  def __getitem__(self, index):
    assert index < self.len
    return self.xs[index], self.ys[index]
  
  def __len__(self):
    return self.len

traindata = Traindata()
testdata = Testdata()
trainloader = DataLoader(traindata, batch_size = batchsize, shuffle = True, num_workers = 2)
testloader = DataLoader(testdata, shuffle = False, batch_size = batchsize)

In [ ]:
def train(epoch):
  running_loss = 0.0
  for batch_idx, data in enumerate(trainloader, 0):
    inputs, target = data
    target = OneHotCategorical(target)
    optimizer.zero_grad()

    # forward + backward + update
    outputs = model(inputs)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if batch_idx % 300 == 299:
      print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
      running_loss = 0.0

In [ ]:
def test():
  correct = 0
  total = 0
  with torch.no_grad():
    for data in testloader:
      images, labels = data
      outputs = model(images)
      _, predicted = torch.max(outputs.data, dim=1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
    print('Accuracy on test set: %d %%' % (100 * correct / total))

In [ ]:
if __name__ == '__main__':
  for epoch in range(10):
    train(epoch)
    test()

KeyError: ignored

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
data = x_train
labels = to_categorical(y_train, 2)

In [ ]:
data

array([[-0.39674768,  0.10702434, -0.22858253, ..., -0.72363347,
         0.33960286,  0.3672473 ],
       [ 0.04542475,  0.07650051,  0.2838936 , ..., -0.5203407 ,
         0.4136464 ,  0.4638726 ],
       [-0.20207046,  0.08581089,  0.17691663, ..., -0.21022011,
         0.29110318, -0.00850555],
       ...,
       [-0.27130172, -0.37212682,  0.12394369, ..., -0.42785767,
         0.07369038,  0.40573177],
       [-0.33609942,  0.05083197,  0.04086212, ..., -0.3342563 ,
         0.04610396,  0.08212136],
       [ 0.05424136,  0.00892559,  0.30004242, ..., -0.5349041 ,
         0.38205302,  0.2552742 ]], dtype=float32)

In [ ]:
from keras.layers import Dropout
from official.nlp import optimization 
model = Sequential()
model.add(Dropout(0.1))
model.add(Dense(1, activation= None, input_dim=768))


epochs = 5
steps_per_epoch = 200
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)
init_lr = 3e-5
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

model.fit(data,labels,epochs=10,batch_size=32)
predictions = model.predict(data)

ModuleNotFoundError: ignored

In [ ]:
predictions

array([[0.52523816],
       [0.4807741 ],
       [0.47772846],
       ...,
       [0.6281905 ],
       [0.404766  ],
       [0.56619245]], dtype=float32)

In [ ]:
official.nlp

NameError: ignored